In [4]:
import torch
from diffusers import DiffusionPipeline
import gc

from diffusers.schedulers.scheduling_dpmsolver_multistep import DPMSolverMultistepScheduler
from PIL import Image
import os


In [5]:
def check_gpu_memory_detailed():
    if not torch.cuda.is_available():
        print("CUDA不可用")
        return
    device = torch.cuda.current_device()
    props = torch.cuda.get_device_properties(device)
    
    # 1. 硬件总显存
    total_gb = props.total_memory / (1024**3)
    # 2. 已主动分配的显存（模型/张量占用）
    allocated_gb = torch.cuda.memory_allocated(device) / (1024**3)
    # 3. 缓存显存（PyTorch预留但未使用的“闲置缓存”）
    cached_gb = torch.cuda.memory_reserved(device) / (1024**3)
    # 4. 实际可用显存 = 总显存 - 已分配 - 缓存
    actual_free_gb = total_gb - allocated_gb - cached_gb
    
    print(f"GPU型号：{props.name}")
    print(f"硬件总显存：{total_gb:.2f} GB")
    print(f"已分配显存（模型/张量）：{allocated_gb:.2f} GB")
    print(f"缓存显存（PyTorch预留）：{cached_gb:.2f} GB")
    print(f"实际可立即使用的显存：{actual_free_gb:.2f} GB")


In [6]:
# 模型选择：使用了 Stable Diffusion v1.5，这是一个平衡了质量和速度的模型，适合移动版 GPU 运行
# 优化设置：
# 使用 FP16 精度减少显存占用
# 集成了 DPMSolver 调度器加速生成过程
# 支持 xFormers 加速（需要额外安装：pip install xformers）
# 禁用梯度计算减少显存使用
# 参数调整：
# 对于 RTX 5090 移动版，建议初始分辨率设置为 768x512 或 512x512
# 推理步数 25-30 步可以在质量和速度间取得平衡
# 引导尺度 7.5 左右可以很好地遵循提示词

# 如果想尝试更先进的模型，可以将model_id改为：
# "stabilityai/stable-diffusion-2-1"（更高质量）
# "runwayml/stable-diffusion-inpainting"（支持图像修复）
# 若遇到显存不足问题，可以：
# 降低图像分辨率
# 减少推理步数
# 添加pipe.enable_attention_slicing()启用注意力切片
# 对于更快的生成速度，可以尝试：
# 使用 "CompVis/stable-diffusion-v1-4" 等较小模型
# 减少推理步数至 20 步
def generate_image(prompt, output_path="output.png", model_id="black-forest-labs/FLUX.1-dev", 
                   width=512, height=512, num_inference_steps=25, guidance_scale=7.5):
    """
    使用Stable Diffusion生成图像
    
    参数:
        prompt: 文本提示词
        output_path: 输出图像路径
        model_id: 模型ID
        width: 生成图像宽度
        height: 生成图像高度
        num_inference_steps: 推理步数
        guidance_scale: 引导尺度，值越大越遵循提示词
    """
    # 检查CUDA是否可用
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"使用设备: {device}")
    
    if device != "cuda":
        print("警告: 未检测到CUDA，将使用CPU运行，速度会很慢")
        return
    # 加载模型并配置调度器
    pipe = DiffusionPipeline.from_pretrained("stabilityai/sd-turbo", 
    torch_dtype=torch.float16,  # 半精度（22GB显存可不用，但加了更高效）
    use_safetensors=True,       # 优先用safetensors（比ckpt更稳定）
    safety_checker=None         # 保持你的需求
    )
    
    # 使用DPMSolver调度器加速生成
    pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
    
    # 移动模型到GPU
    pipe = pipe.to(device)
    
    # 启用xFormers加速（如果安装了xFormers）
    try:
        pipe.enable_xformers_memory_efficient_attention()
        print("已启用xFormers加速")
    except:
        print("未安装xFormers，将使用默认注意力机制")
    
    # 生成图像
    with torch.no_grad():  # 禁用梯度计算节省显存
        image = pipe(
            prompt,
            width=width,
            height=height,
            num_inference_steps=num_inference_steps,
            guidance_scale=guidance_scale
        ).images[0]
    
    # 保存图像
    image.save(output_path)
    print(f"图像已保存至: {output_path}")
    print(torch.cuda.memory_summary(device=None, abbreviated=False))
    return image


In [ ]:
torch.cuda.empty_cache()
gc.collect()

prompt = "a beautiful sunset over the mountains, detailed, vibrant colors, 8k resolution"

# 创建输出目录
os.makedirs("outputs_images", exist_ok=True)

# 生成图像
generate_image(
    prompt=prompt,
    model_id="black-forest-labs/FLUX.1-dev", 
    output_path="outputs_images/sunset.png",
    width=768,  # RTX 5090移动版可以流畅处理这个分辨率
    height=512,
    num_inference_steps=30
)

check_gpu_memory_detailed()

使用设备: cuda


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


未安装xFormers，将使用默认注意力机制


  0%|          | 0/30 [00:00<?, ?it/s]

图像已保存至: outputs_images/sunset.png
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   7450 MiB |   8364 MiB | 369423 MiB | 361973 MiB |
|       from large pool |   7251 MiB |   8165 MiB | 356019 MiB | 348768 MiB |
|       from small pool |    198 MiB |    207 MiB |  13403 MiB |  13205 MiB |
|---------------------------------------------------------------------------|
| Active memory         |   7450 MiB |   8364 MiB | 369423 MiB | 361973 MiB |
|       from large pool |   72